# Tratamento dos Dados (Silver)

## Objetivos
- Padronizar nomes das colunas para snake_case
- Tratar valores nulos e inconsistentes
- Converter tipos de dados apropriadamente
- Extrair colunas úteis para análises gráficas
- Preparar dados para visualizações (pizza, regressão linear, boxplot)

## Estrutura dos Dados Originais
O dataset contém informações sobre escolas brasileiras do INEP com 19 colunas principais:
- Informações geográficas (UF, Município, Localização, Latitude, Longitude)
- Características administrativas (Dependência, Categoria, Porte)
- Informações educacionais (Etapas de Ensino, Modalidades)
- Status operacional (Restrição de Atendimento, Regulamentação)

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, trim, when, split, size, coalesce, lit, regexp_replace, lower, regexp_extract
from pyspark.sql.types import IntegerType, DoubleType, StringType
import pyspark.sql.functions as F

# Inicializar Spark Session
spark = SparkSession.builder.appName("inep_schools_analysis").getOrCreate()

# Carregar dados bronze
df = spark.read.parquet("../raw/bronze_escolas.parquet")

print("=== ESTRUTURA INICIAL DOS DADOS ===")
print(f"Total de registros: {df.count():,}")
print(f"Número de colunas: {len(df.columns)}")
df.printSchema()


=== ESTRUTURA INICIAL DOS DADOS ===


Total de registros: 212,386
Número de colunas: 19
root
 |-- Restrição de Atendimento: string (nullable = true)
 |-- Escola: string (nullable = true)
 |-- Código INEP: string (nullable = true)
 |-- UF: string (nullable = true)
 |-- Município: string (nullable = true)
 |-- Localização: string (nullable = true)
 |-- Localidade Diferenciada: string (nullable = true)
 |-- Categoria Administrativa: string (nullable = true)
 |-- Endereço: string (nullable = true)
 |-- Telefone: string (nullable = true)
 |-- Dependência Administrativa: string (nullable = true)
 |-- Categoria Escola Privada: string (nullable = true)
 |-- Conveniada Poder Público: string (nullable = true)
 |-- Regulamentação pelo Conselho de Educação: string (nullable = true)
 |-- Porte da Escola: string (nullable = true)
 |-- Etapas e Modalidade de Ensino Oferecidas: string (nullable = true)
 |-- Outras Ofertas Educacionais: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true

In [3]:
# === LIMPEZA E PADRONIZAÇÃO DOS DADOS ===

print("=== ANÁLISE DE VALORES NULOS ===")
# Verificar valores nulos por coluna
null_counts = df.select([F.count(F.when(F.col(c).isNull() | (F.col(c) == ""), c)).alias(c) for c in df.columns]).collect()[0]
for col_name in df.columns:
    null_count = null_counts[col_name]
    if null_count > 0:
        print(f"{col_name}: {null_count:,} valores nulos/vazios")

# Renomear colunas para snake_case
df_clean = df.withColumnRenamed("Restrição de Atendimento", "restricao_atendimento") \
             .withColumnRenamed("Escola", "nome_escola") \
             .withColumnRenamed("Código INEP", "codigo_inep") \
             .withColumnRenamed("UF", "uf") \
             .withColumnRenamed("Município", "municipio") \
             .withColumnRenamed("Localização", "localizacao") \
             .withColumnRenamed("Localidade Diferenciada", "localidade_diferenciada") \
             .withColumnRenamed("Categoria Administrativa", "categoria_administrativa") \
             .withColumnRenamed("Endereço", "endereco") \
             .withColumnRenamed("Telefone", "telefone") \
             .withColumnRenamed("Dependência Administrativa", "dependencia_administrativa") \
             .withColumnRenamed("Categoria Escola Privada", "categoria_escola_privada") \
             .withColumnRenamed("Conveniada Poder Público", "conveniada_poder_publico") \
             .withColumnRenamed("Regulamentação pelo Conselho de Educação", "regulamentacao_conselho") \
             .withColumnRenamed("Porte da Escola", "porte_escola") \
             .withColumnRenamed("Etapas e Modalidade de Ensino Oferecidas", "etapas_modalidades") \
             .withColumnRenamed("Outras Ofertas Educacionais", "outras_ofertas") \
             .withColumnRenamed("Latitude", "latitude") \
             .withColumnRenamed("Longitude", "longitude")

print("\n=== COLUNAS RENOMEADAS PARA SNAKE_CASE ===")
df_clean.printSchema()


=== ANÁLISE DE VALORES NULOS ===


Telefone: 42,433 valores nulos/vazios
Regulamentação pelo Conselho de Educação: 31,321 valores nulos/vazios
Porte da Escola: 31,321 valores nulos/vazios
Etapas e Modalidade de Ensino Oferecidas: 33,163 valores nulos/vazios
Outras Ofertas Educacionais: 153,326 valores nulos/vazios

=== COLUNAS RENOMEADAS PARA SNAKE_CASE ===
root
 |-- restricao_atendimento: string (nullable = true)
 |-- nome_escola: string (nullable = true)
 |-- codigo_inep: string (nullable = true)
 |-- uf: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- localizacao: string (nullable = true)
 |-- localidade_diferenciada: string (nullable = true)
 |-- categoria_administrativa: string (nullable = true)
 |-- endereco: string (nullable = true)
 |-- telefone: string (nullable = true)
 |-- dependencia_administrativa: string (nullable = true)
 |-- categoria_escola_privada: string (nullable = true)
 |-- conveniada_poder_publico: string (nullable = true)
 |-- regulamentacao_conselho: string (nullable = tru

In [4]:
# === CONVERSÃO DE TIPOS E TRATAMENTO DE VALORES NULOS ===

# Converter coordenadas para double, tratando valores inválidos
df_clean = df_clean.withColumn("latitude", 
    when(col("latitude").rlike("^-?\\d+\\.?\\d*$"), col("latitude").cast(DoubleType()))
    .otherwise(None)
).withColumn("longitude", 
    when(col("longitude").rlike("^-?\\d+\\.?\\d*$"), col("longitude").cast(DoubleType()))
    .otherwise(None)
)

# Limpar strings removendo espaços extras e normalizando
string_columns = ["restricao_atendimento", "nome_escola", "uf", "municipio", 
                 "localizacao", "categoria_administrativa", "dependencia_administrativa",
                 "porte_escola", "etapas_modalidades"]

for col_name in string_columns:
    df_clean = df_clean.withColumn(col_name, trim(col(col_name)))

# Tratar valores "Não Informado" como nulos
df_clean = df_clean.withColumn("categoria_escola_privada", 
    when(col("categoria_escola_privada") == "Não Informado", None)
    .otherwise(col("categoria_escola_privada"))
)

# Filtrar apenas escolas com coordenadas válidas
df_clean = df_clean.filter(
    col("latitude").isNotNull() & 
    col("longitude").isNotNull() &
    (col("latitude") != 0) & 
    (col("longitude") != 0)
)

print(f"=== DADOS APÓS LIMPEZA ===")
print(f"Registros válidos: {df_clean.count():,}")
print(f"Registros removidos: {df.count() - df_clean.count():,}")


=== DADOS APÓS LIMPEZA ===


Registros válidos: 156,423
Registros removidos: 55,963


In [5]:
# === CRIAÇÃO DE COLUNAS DERIVADAS PARA ANÁLISES ===

# 1. Contar número de etapas/modalidades oferecidas
df_clean = df_clean.withColumn("etapas_list", 
    split(regexp_replace(col("etapas_modalidades"), r",\\s*", ","), ",")
).withColumn("num_etapas", size(col("etapas_list")))

# 2. Mapear porte da escola para valores numéricos
def map_porte_to_numeric(porte):
    if porte is None:
        return None
    porte_lower = porte.lower().strip()
    if "pequeno" in porte_lower or "até 50" in porte_lower:
        return 1
    elif "médio" in porte_lower or "51" in porte_lower or "201" in porte_lower or "501" in porte_lower:
        return 2
    elif "grande" in porte_lower or "1000" in porte_lower:
        return 3
    else:
        return None

from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

map_porte_udf = udf(map_porte_to_numeric, IntegerType())
df_clean = df_clean.withColumn("porte_numerico", map_porte_udf(col("porte_escola")))

# 3. Criar flag para escolas rurais vs urbanas
df_clean = df_clean.withColumn("is_rural", 
    when(col("localizacao") == "Rural", 1).otherwise(0)
)

# 4. Criar flag para escolas públicas vs privadas
df_clean = df_clean.withColumn("is_publica", 
    when(col("dependencia_administrativa").isin(["Estadual", "Municipal", "Federal"]), 1)
    .otherwise(0)
)

# 5. Extrair região do Brasil baseada na UF
def get_regiao(uf):
    regioes = {
        'AC': 'Norte', 'AM': 'Norte', 'AP': 'Norte', 'PA': 'Norte', 'RO': 'Norte', 'RR': 'Norte', 'TO': 'Norte',
        'AL': 'Nordeste', 'BA': 'Nordeste', 'CE': 'Nordeste', 'MA': 'Nordeste', 'PB': 'Nordeste', 
        'PE': 'Nordeste', 'PI': 'Nordeste', 'RN': 'Nordeste', 'SE': 'Nordeste',
        'DF': 'Centro-Oeste', 'GO': 'Centro-Oeste', 'MT': 'Centro-Oeste', 'MS': 'Centro-Oeste',
        'ES': 'Sudeste', 'MG': 'Sudeste', 'RJ': 'Sudeste', 'SP': 'Sudeste',
        'PR': 'Sul', 'RS': 'Sul', 'SC': 'Sul'
    }
    return regioes.get(uf, 'Outro')

get_regiao_udf = udf(get_regiao, StringType())
df_clean = df_clean.withColumn("regiao", get_regiao_udf(col("uf")))

print("=== COLUNAS DERIVADAS CRIADAS ===")
print("Colunas adicionadas:")
print("- num_etapas: número de etapas/modalidades oferecidas")
print("- porte_numerico: porte da escola (1=Pequeno, 2=Médio, 3=Grande)")
print("- is_rural: flag para escolas rurais (1=Rural, 0=Urbana)")
print("- is_publica: flag para escolas públicas (1=Pública, 0=Privada)")
print("- regiao: região do Brasil baseada na UF")


=== COLUNAS DERIVADAS CRIADAS ===
Colunas adicionadas:
- num_etapas: número de etapas/modalidades oferecidas
- porte_numerico: porte da escola (1=Pequeno, 2=Médio, 3=Grande)
- is_rural: flag para escolas rurais (1=Rural, 0=Urbana)
- is_publica: flag para escolas públicas (1=Pública, 0=Privada)
- regiao: região do Brasil baseada na UF


In [ ]:
# === SELEÇÃO DE COLUNAS E SALVAMENTO DOS DADOS TRATADOS ===

# Selecionar apenas as colunas necessárias para as análises gráficas
colunas_analise = [
    "codigo_inep", "nome_escola", "uf", "municipio", "regiao",
    "localizacao", "is_rural", "dependencia_administrativa", "is_publica",
    "porte_escola", "porte_numerico", "etapas_modalidades", "num_etapas",
    "latitude", "longitude", "restricao_atendimento"
]

df_final = df_clean.select(*colunas_analise)

print("=== DADOS FINAIS PARA ANÁLISE ===")
print(f"Total de registros: {df_final.count():,}")
print(f"Colunas selecionadas: {len(colunas_analise)}")
print("\nColunas incluídas:")
for col in colunas_analise:
    print(f"- {col}")

# Salvar dados tratados (Silver Layer)
df_final.write.mode("overwrite").parquet("escolas_silver.parquet")

print("\n=== DADOS SALVOS COM SUCESSO ===")
print("Arquivo salvo: escolas_silver.parquet")
print("Dados prontos para análise no notebook Gold!")


=== DADOS FINAIS PARA ANÁLISE ===


Total de registros: 156,423
Colunas selecionadas: 16

Colunas incluídas:
- codigo_inep
- nome_escola
- uf
- municipio
- regiao
- localizacao
- is_rural
- dependencia_administrativa
- is_publica
- porte_escola
- porte_numerico
- etapas_modalidades
- num_etapas
- latitude
- longitude
- restricao_atendimento


25/09/29 20:30:10 WARN MemoryManager: Total allocation exceeds 95,00% (1.020.054.720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers



=== DADOS SALVOS COM SUCESSO ===
Arquivo salvo: processed/escolas_silver.parquet
Dados prontos para análise no notebook Gold!


In [9]:
from pyspark.sql.functions import split, size, coalesce, lit, col, udf
from pyspark.sql.types import IntegerType

# Usar o DataFrame já processado (df_final) em vez de df original
df = df_final

df = df.withColumn("etapas_list", split(col("etapas_modalidades"), r"[;,]"))
df = df.withColumn("count_etapas", size(col("etapas_list")))

mapping_porte = {"Pequeno":1, "Médio":2, "Grande":3}

# UDF simples
def map_porte(v):
    if v is None: return None
    v = v.strip().title()
    return mapping_porte.get(v, None)

map_porte_udf = udf(map_porte, IntegerType())
df = df.withColumn("porte_num", map_porte_udf(col("porte_escola")))
